<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/04_mcp/04_mcp_with_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install -q mcp openai nest_asyncio

In [26]:
import nest_asyncio
import sys
import os

In [27]:
# 1. Apply the patch allows nested async loops in Colab
nest_asyncio.apply()

In [42]:
%%writefile converter.py
from mcp.server.fastmcp import FastMCP
import logging

# 1. Setup Server (Silence info logs)
logging.basicConfig(level=logging.ERROR)
mcp = FastMCP("UnitConverter")

# 2. Define Tool: Feet to Meters
@mcp.tool()
def feet_to_meters(feet: float) -> float:
    """Converts a length from feet to meters."""
    return feet * 0.3048

# 3. Define Tool: Meters to Feet
@mcp.tool()
def meters_to_feet(meters: float) -> float:
    """Converts a length from meters to feet."""
    return meters / 0.3048

if __name__ == "__main__":
    mcp.run()

Writing converter.py


In [43]:
import asyncio
import sys
import nest_asyncio
from openai import OpenAI
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Apply Colab async fix
nest_asyncio.apply()

class SimpleMCPAgent:
    def __init__(self, server_script, api_key):
        self.server_script = server_script
        self.client = OpenAI(api_key=api_key)

    async def process_query(self, user_query):
        """Connects to server, gets tools, asks AI, and runs the tool."""

        # 1. SERVER CONFIG (The "-u" flag is vital for Colab!)
        server_params = StdioServerParameters(
            command=sys.executable,
            args=["-u", self.server_script],
        )

        print(f"🔌 Connecting to {self.server_script}...")

        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()

                # 2. DISCOVER TOOLS (Dynamic!)
                tools = await session.list_tools()
                openai_tools = [self._convert_tool(t) for t in tools.tools]
                print(f"🛠️  Agent found these tools: {[t.name for t in tools.tools]}")

                # 3. THINK (Ask OpenAI)
                messages = [{"role": "user", "content": user_query}]
                response = self.client.chat.completions.create(
                    model="gpt-4o",
                    messages=messages,
                    tools=openai_tools
                )

                # 4. DECIDE & ACT
                tool_call = response.choices[0].message.tool_calls[0] if response.choices[0].message.tool_calls else None

                if tool_call:
                    func_name = tool_call.function.name
                    func_args = eval(tool_call.function.arguments)

                    print(f"🤖 Agent decided to call: {func_name}({func_args})")

                    # Execute on Server
                    result = await session.call_tool(func_name, arguments=func_args)
                    print(f"✅ Tool Result: {result.content[0].text}")
                    return result.content[0].text
                else:
                    print("🤷 Agent didn't need a tool.")
                    return response.choices[0].message.content

    def _convert_tool(self, tool):
        """Helper to format MCP tools for OpenAI"""
        return {
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.inputSchema
            }
        }

# --- usage ---
print("Agent Class Ready.")

Agent Class Ready.


In [44]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
# 1. SETUP: Put your key here
API_KEY = OPENAI_API_KEY  # <--- REPLACE WITH YOUR KEY

# 2. INITIALIZE
agent = SimpleMCPAgent("converter.py", api_key=API_KEY)

# 3. ASK A QUESTION
query = "I have a 100 foot rope. How many meters is that?"
result = await agent.process_query(query)